In [98]:
import numpy as np
import itertools
import random
import itertools as it

Часть 1. Расширенный код Голея

In [99]:
B_Goley = np.array([
    [1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
    [1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1],
    [0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1],
    [1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1],
    [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1],
    [1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1],
    [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
    [0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1],
    [1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1],
    [0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])


In [100]:
print(len(B_Goley))

12


In [101]:
# функция формирования порождающей матрицы для РКГ

def func_G_Goley (n, k, x):
  G = np.zeros((k,n), dtype = int)
  I = np.eye (k)
  for i in range(k):
    for j in range(k):
      G[i,j] = I[i,j]
    for m in range(n-k):
      G[i,k + m] = x[i,m]
  return G


# функция формирования проверочной матрицы ДЛЯ РКГ

def func_H_Goley (G):
  n = len(G)
  k = len(G.T)
  H = np.zeros((k,k-n), dtype = int)
  x_h = np.zeros((n,k-n), dtype = int)
  I = np.eye(k-n)
  for i in range(n):
    for j in range(k-n):
      H[i,j] = G[i, j + n]
  for i in range(n):
    for j in range(k-n):
      H[i + n,j] = I[i,j]
  return H

# Получение синдрома ошибки
def get_syndroms(G, H):
    return G.dot(H) % 2


# Генерация слова с указанным количеством ошибок
def generate_code_words(G, error_count):
    u = np.zeros(len(G), dtype=int)
    for i in range(len(u)):
        u[i] = random.randint(0, 1)
    print("Исходное слово", u)
    u = u.dot(G)
    u %= 2
    err_arr = np.full(error_count, len(u) + 1, dtype=int)
    for k in range(error_count):
        mistake_pos = random.randint(0, len(u) - 1)
        while mistake_pos in err_arr:
            mistake_pos = random.randint(0, len(u) - 1)
        err_arr[k] = mistake_pos
        u[mistake_pos] += 1
        u[mistake_pos] %= 2
    return u

# Исправление одинарной ошибки
def fix_one_bag(H, syndrome, message):
    index = -1
    for j in range(len(H)):
        if np.array_equal(syndrome, H[j]):
            index = j
    if index == -1:
        print("Такого синдрома нет в матрице синдромов", 'n')
    else:
        message[index] = (message[index] + 1) % 2
    return message


# Исправление двойной ошибки
def fix_two_bag(H, error_syndrome, message):
    idx1 = -1
    idx2 = -1
    for row in range(len(H)):
        if np.array_equal(error_syndrome, H[row]):
            idx1 = row
            break
        for col in range(row + 1, len(H)):
            if np.array_equal(error_syndrome, H[row] + H[col]):
                idx1 = row
                idx2 = col
                break
        if idx1 >= 0:
            break
    if idx1 == -1:
        print("Такого синдрома нет в матрице синдромов", 'n')
    else:
        message[idx1] = (message[idx1] + 1) %2
        if idx2 != -1:
            message[idx2] = (message[idx2] + 1) %2
    return message


# Исправление тройной ошибки
def fix_three_bag(H, syndrome, message):
    index1 = -1
    index2 = -1
    index3 = -1
    for i in range(len(H)):
        if np.array_equal(syndrome, H[i]):
            index1 = i
            break
        for j in range(i + 1, len(H)):
            if np.array_equal(syndrome, H[i] + H[j]):
                index1 = i
                index2 = j
                break
            for k in range(j + 1, len(H)):
                if np.array_equal(syndrome, H[i] + H[j] + H[k]):
                    index1 = i
                    index2 = j
                    index3 = k
                    break
            if index1 >= 0:
                break
        if index1 >= 0:
            break
    if index1 == -1:
        print("Такого синдрома нет в матрице синдромов", 'n')
    else:
        message[index1] ^= 1
        if index2 != -1:
            message[index2] ^= 1
        if index3 != -1:
            message[index3] ^= 1

    return message


# Исправление четверной ошибки
def fix_four_bag(H, syndrome, message):
    index1 = -1
    index2 = -1
    index3 = -1
    index4 = -1

    for i in range(len(H)):
        if np.array_equal(syndrome, H[i]):
            index1 = i
            break

        for j in range(i + 1, len(H)):
            if np.array_equal(syndrome, H[i] + H[j]):
                index1 = i
                index2 = j
                break

            for k in range(j + 1, len(H)):
                if np.array_equal(syndrome, H[i] + H[j] + H[k]):
                    index1 = i
                    index2 = j
                    index3 = k
                    break

                for l in range(k + 1, len(H)):
                    if np.array_equal(syndrome, H[i] + H[j] + H[k] + H[l]):
                        index1 = i
                        index2 = j
                        index3 = k
                        index4 = l
                        break
                if index1 >= 0:
                    break
            if index1 >= 0:
                break
        if index1 >= 0:
            break

    if index1 != -1:
        message[index1] ^= 1
        if index2 != -1:
            message[index2] ^= 1
        if index3 != -1:
            message[index3] ^= 1
        if index4 != -1:
            message[index4] ^= 1
    else:
      print("Такого синдрома нет в матрице синдромов", '\n')
    return message

In [105]:
G = func_G_Goley(24, 12, B_Goley)
print("Порождающая матрица:", '\n', G, '\n')

H = func_H_Goley(G)
print("Проверочная матрица H", '\n', H, '\n')

print("__________")

count = 3  #количество ошибок в слове
word_with_mistake = generate_code_words(G, count)
S = get_syndroms(word_with_mistake, H)

match count:
  case 1:
    print("Кодовое слово с одной ошибкой", '\n', word_with_mistake, '\n')
    print("Синдром для кодового слова с ошибкой", '\n', S, '\n')
    correct_word_mistake = fix_one_bag(H, S, word_with_mistake)

  case 2:
    print("Кодовое слово с двумя ошибками", '\n', word_with_mistake, '\n')
    print("Синдром для кодового слова с ошибкой", '\n', S, '\n')
    correct_word_mistake = fix_two_bag(H, S, word_with_mistake)

  case 3:
    print("Кодовое слово с тремя ошибками", '\n', word_with_mistake, '\n')
    print("Синдром для кодового слова с ошибкой", '\n', S, '\n')
    correct_word_mistake = fix_three_bag(H, S, word_with_mistake)

  case 4:
    print("Кодовое слово с четыремя ошибками", '\n', word_with_mistake, '\n')
    print("Синдром для кодового слова с ошибкой", '\n', S, '\n')
    correct_word_mistake = fix_four_bag(H, S, word_with_mistake)

print("Исправленное кодовое слово c одной ошибкой", '\n', correct_word_mistake, '\n')

test = np.dot(correct_word_mistake, H) % 2
print("Проверка", '\n', test, '\n')





Порождающая матрица: 
 [[1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 1 0 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 1 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 1]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 1 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0]] 

Проверочная матрица H 
 [[1 1 0 1 1 1 0 0 0 1 0 1]
 [1 0 1 1 1 0 0 0 1 0 1 1]
 [0 1 1 1 0 0 0 1 0 1 1 1]
 [1 1 1 0 0 0 1 0 1 1 0 1]
 [1 1 0 0 0 1 0 1 1 0 1 1]
 [1 0 0 0 1 0 1 1 0 1 1 1]
 [0 0 0 1 0 1 1 0 1 1 1 1]
 [0 0 1 0 1 1 0 1 1 1 0 1]
 [0 1 0 1 1 0 1 1 1 0 0 1]
 [1 0 1 1 0 1 1 1 0 0 0 1]
 [0 1 1 0 1 1 1 0 0 0 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0]
 [1 0 0 0 0 0 

In [103]:

# H для произведения Кронекера
H_kron = np.array([[1, 1],
                   [1, -1]])

# Порождающая матрица кода Рида-Маллера
def generate_rid_maller(row, col):
    if 0 < row < col:
        upper_matrix = generate_rid_maller(row, col - 1)
        lower_matrix = generate_rid_maller(row - 1, col - 1)
        left_part = np.concatenate([upper_matrix, np.zeros((len(lower_matrix), len(upper_matrix[0])), int)])
        right_part = np.concatenate([upper_matrix, lower_matrix])
        return np.concatenate([left_part, right_part], axis=1)
    elif row == 0:
        return np.ones((1, 2 ** col), dtype=int)
    elif row == col:
        upper_block = generate_rid_maller(row - 1, col)
        bottom_block = np.zeros((1, 2 ** col), dtype=int)
        bottom_block[0][-1] = 1
        return np.concatenate([upper_block, bottom_block])

# Проверочная матрица кода Рида-Маллера
def check_rid_maller(H, index, col):
    left_kron = np.kron(np.eye(2 ** (col - index), dtype=int), H)
    right_kron = np.kron(left_kron, np.eye(2 ** (index - 1), dtype=int))
    return right_kron


def decode_message(matrix, rid_matrix, mistakes_count):
    codewords = generate_code_words(rid_matrix, mistakes_count)
    codewords = np.where(codewords == 0, -1, codewords)

    check_matrix_1 = check_rid_maller(matrix, 1, 3)
    check_matrix_2 = check_rid_maller(matrix, 2, 3)
    check_matrix_3 = check_rid_maller(matrix, 3, 3)

    intermediate_result_1 = codewords.dot(check_matrix_1)
    intermediate_result_2 = intermediate_result_1.dot(check_matrix_2)
    final_result = intermediate_result_2.dot(check_matrix_3)

    best_value = np.max(final_result)
    best_index = np.argmax(final_result)

    binary_representation = np.binary_repr(best_index, width=3)[::-1]
    binary_prefix = "1" if best_value > 0 else "0"
    binary_message = binary_prefix + binary_representation

    print("Количество ошибок:", mistakes_count)
    print("Максимальное значение:", best_value)
    print("Декодированное сообщение:", binary_message)


def decode_message_2(H_matrix, risk_factor, error_count):
    code_words = generate_code_words(risk_factor, error_count)
    code_words = np.where(code_words == 0, -1, code_words)

    h_1 = check_rid_maller(H_matrix, 1, 4)
    h_2 = check_rid_maller(H_matrix, 2, 4)
    h_3 = check_rid_maller(H_matrix, 3, 4)
    h_4 = check_rid_maller(H_matrix, 4, 4)

    result_1 = code_words.dot(h_1)
    result_2 = result_1.dot(h_2)
    result_3 = result_2.dot(h_3)
    final_result = result_3.dot(h_4)

    max_value = np.max(final_result)
    max_index = np.argmax(final_result)

    binary_output = np.binary_repr(max_index, width=4)[::-1]
    if max_value > 0:
        binary_output = "1" + binary_output
    else:
        binary_output = "0" + binary_output

    print("Количество ошибок:", error_count)
    print("Максимальное значение:", max_value)
    print("Декодированное сообщение:", binary_output)


In [104]:
r_1 = 1
m_1 = 3
RM = generate_rid_maller(r_1, m_1)
print("Порождающая матрица Рида Маллера", '\n', RM, '\n')

decode_message(H_kron, RM, 1)
decode_message(H_kron, RM, 2)

r_2 = 1
m_2 = 4
RM = generate_rid_maller(r_2, m_2)
print("Порождающая матрица Рида Маллера (1,4)", '\n', RM, '\n')

decode_message_2(H_kron, RM, 1)
decode_message_2(H_kron, RM, 2)
decode_message_2(H_kron, RM, 3)
decode_message_2(H_kron, RM, 4)

Порождающая матрица Рида Маллера 
 [[1 1 1 1 1 1 1 1]
 [0 1 0 1 0 1 0 1]
 [0 0 1 1 0 0 1 1]
 [0 0 0 0 1 1 1 1]] 

Исходное слово [1 1 1 1]
Количество ошибок: 1
Максимальное значение: 6
Декодированное сообщение: 1111
Исходное слово [1 0 0 0]
Количество ошибок: 2
Максимальное значение: 4
Декодированное сообщение: 1000
Порождающая матрица Рида Маллера (1,4) 
 [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
 [0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
 [0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]] 

Исходное слово [0 1 0 1 0]
Количество ошибок: 1
Максимальное значение: 2
Декодированное сообщение: 10010
Исходное слово [0 1 0 1 1]
Количество ошибок: 2
Максимальное значение: 4
Декодированное сообщение: 11000
Исходное слово [1 0 1 0 0]
Количество ошибок: 3
Максимальное значение: 10
Декодированное сообщение: 10100
Исходное слово [0 1 1 0 1]
Количество ошибок: 4
Максимальное значение: 8
Декодированное сообщение: 11110
